In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
# Pandas ayarları: Tüm sütunları görmek istiyorum, gizleme yapma
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [20]:
df = pd.read_csv('clean_data.csv')

In [21]:
df.head()

,brand,model,variant,year,km,color,price,city,engine_volume,engine_tech,package
0,audi,a1,1.4 TFSI Attraction,2011,155000,White,625000,İstanbul,1.4,tfsi,attraction
1,audi,a1,1.6 TDI Ambition,2011,252000,Black,625000,Aydın,1.6,tdi,ambition
2,audi,a1,1.4 TFSI Attraction,2012,203000,Black,665000,Zonguldak,1.4,tfsi,attraction
3,audi,a1,1.6 TDI Ambition,2013,270000,White,835000,İstanbul,1.6,tdi,ambition
4,audi,a1,1.6 TDI Attraction,2011,228000,Black,670000,Ankara,1.6,tdi,attraction


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218579 entries, 0 to 218578
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   brand          218579 non-null  object 
 1   model          218579 non-null  object 
 2   variant        218579 non-null  object 
 3   year           218579 non-null  int64  
 4   km             218579 non-null  int64  
 5   color          218579 non-null  object 
 6   price          218579 non-null  int64  
 7   city           218579 non-null  object 
 8   engine_volume  218579 non-null  float64
 9   engine_tech    218579 non-null  object 
 10  package        218579 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 18.3+ MB


In [23]:
# ---------------------------------------------------------
# HAMLE 0: EKSİK HALKA (YAKIT TİPİ DÖNÜŞÜMÜ)
# ---------------------------------------------------------
fuel_map = {
    'tdi': 'Diesel', 'cdti': 'Diesel', 'dci': 'Diesel', 'hdi': 'Diesel', 
    'bluehdi': 'Diesel', 'multijet': 'Diesel', 'crdi': 'Diesel', 'cdi': 'Diesel',
    'tfsi': 'Gasoline', 'tsi': 'Gasoline', 'vtec': 'Gasoline', 'gdi': 'Gasoline', 
    'ecoboost': 'Gasoline', 'turbo': 'Gasoline',
    'hybrid': 'Hybrid', 'electric': 'Electric',
    'lpg': 'LPG'
}

print("Yakıt Türleri (fuel_type) oluşturuluyor")
df['fuel_type'] = df['engine_tech'].map(fuel_map).fillna('Gasoline')

# ---------------------------------------------------------
# HAMLE 0: VİTES TİPİ (TRANSMISSION) OLUŞTURMA
# ---------------------------------------------------------

def extract_transmission(text):
    text = str(text).lower()
    if any(x in text for x in ['otomatik', 'dsg', 'edc', 'cvt', 'tiptronik', 'stronic', 'eat', 'automatic']):
        return 'Automatic'
    elif 'manuel' in text:
        return 'Manual'
    else:
        return 'Manual' # Varsayılan

print("🛠️ Vites tipleri (transmission) oluşturuluyor...")
df['transmission'] = df['variant'].apply(extract_transmission)

# ---------------------------------------------------------
# HAMLE 1: HEDEF DEĞİŞKEN DÖNÜŞÜMÜ (Log Transformation)
# ---------------------------------------------------------

df['price_log'] = np.log1p(df['price'])
print("Fiyatlar logaritmik olarak (price_log) dönüştürüldü")

# ---------------------------------------------------------
# HAMLE 2: GEREKSİZ YÜKLERİ AT (Feature Selection)
# ---------------------------------------------------------

cols_to_drop = ['engine_tech', 'variant']
df = df.drop(columns = cols_to_drop, errors='ignore')
print("Gereksiz sütunlar (variant, engine_tech) atıldı.")

# ---------------------------------------------------------
# HAMLE 3: ENCODING STRATEJİSİ
# ---------------------------------------------------------
# A. One-Hot Encoding (One-Hot)

ohe_cols = ['fuel_type', 'transmission', 'brand']
df = pd.get_dummies(df, columns = ohe_cols, drop_first=True)
print("One-Hot Encoding tamamlandı.")

# B. Frequency Encoding (Frekans)
freq_cols = ['model', 'city', 'color', 'package']

for col in freq_cols:
    if col in df.columns:
        freq_map = df[col].value_counts(normalize=True)
        df[col + '_freq_enc'] = df[col].map(freq_map)
        df = df.drop(columns=[col])

print("Frekans encoding tamamlandı.")

# ---------------------------------------------------------
# HAMLE 4: KAYIT
# ---------------------------------------------------------

print("-" * 40)
print(f"İşlem Sonrası Boyut: {df.shape}")
print("\nVeri Tipleri Özeti:")
print(df.dtypes.value_counts())

df.to_pickle("processed_data.pkl")
df.to_csv("processed_data.csv", index=False)

print("\n🚀 processed_data.pkl kaydedildi. Notebook 3 için hazırsın!")
print(df.head())


Yakıt Türleri (fuel_type) oluşturuluyor
🛠️ Vites tipleri (transmission) oluşturuluyor...
Fiyatlar logaritmik olarak (price_log) dönüştürüldü
Gereksiz sütunlar (variant, engine_tech) atıldı.
One-Hot Encoding tamamlandı.
Frekans encoding tamamlandı.
----------------------------------------
İşlem Sonrası Boyut: (218579, 34)

Veri Tipleri Özeti:
bool       25
float64     6
int64       3
Name: count, dtype: int64

🚀 processed_data.pkl kaydedildi. Notebook 3 için hazırsın!
   year      km   price  engine_volume  price_log  fuel_type_Gasoline  fuel_type_Hybrid  transmission_Manual  brand_bmw  brand_chevrolet  brand_citroen  brand_dacia  brand_fiat  brand_ford  brand_honda  brand_hyundai  brand_kia  brand_mazda  brand_mercedes  brand_mini  brand_mitsubishi  brand_nissan  brand_opel  brand_peugeot  brand_renault  brand_seat  brand_skoda  brand_toyota  brand_volkswagen  brand_volvo  model_freq_enc  city_freq_enc  color_freq_enc  package_freq_enc
0  2011  155000  625000            1.4  13.345509 

In [27]:
df.columns

Index(['year', 'km', 'price', 'engine_volume', 'price_log', 'fuel_type_Gasoline', 'fuel_type_Hybrid', 'transmission_Manual', 'brand_bmw', 'brand_chevrolet', 'brand_citroen', 'brand_dacia', 'brand_fiat', 'brand_ford', 'brand_honda', 'brand_hyundai', 'brand_kia', 'brand_mazda', 'brand_mercedes', 'brand_mini', 'brand_mitsubishi', 'brand_nissan', 'brand_opel', 'brand_peugeot', 'brand_renault', 'brand_seat', 'brand_skoda', 'brand_toyota', 'brand_volkswagen', 'brand_volvo', 'model_freq_enc',
       'city_freq_enc', 'color_freq_enc', 'package_freq_enc'],
      dtype='object')